In [1]:
# https://github.com/DLR-RM/stable-baselines3/issues/383 (not so useful)
# https://github.com/DLR-RM/stable-baselines3/issues/1349

import os 
os.chdir('/home/giuseppe/Desktop/tesiEnv')

In [2]:
import torch as th
from stable_baselines3 import PPO

import onnx
import onnxruntime as rt
import numpy as np

2023-08-31 19:33:48.381503: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 19:33:48.988294: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Convert Stable Baseline Model to ONNX

In [3]:
class OnnxablePolicyPyTorch2(th.nn.Module):
    def __init__(self, policy):
        super().__init__()
        self.policy = policy

    def forward(self, observation):
        # NOTE: Preprocessing is included, the only thing you need to do
        # is transpose the images if needed so that they are channel first
        # use deterministic=False if you want to export the stochastic policy
        return self.policy(observation, deterministic=False)

In [4]:
model_name = "ts1000000"
model = PPO.load(model_name, device="cpu")

In [5]:
onnx_pytorch2 = OnnxablePolicyPyTorch2(model.policy)
observation_size = model.observation_space.shape
dummy_input = th.randn(1, *observation_size)

In [6]:
th.onnx.export(
    onnx_pytorch2,
    dummy_input,
    "model.onnx",
    opset_version=17,  # neeed a "updated" version of onnx
    input_names=["input"],
    verbose=0
)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



### Load onxx Model

In [3]:
providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']
model_name = 'PPO_Stoc.onnx'

In [4]:
onnx_model = onnx.load(model_name)
onnx.checker.check_model(onnx_model)

In [5]:
ort_sess = rt.InferenceSession(model_name,providers=providers)